# Einstein-Bianchi equations

(work with Edoardo Bonetti)

Components of the Weyl-tensor satisfy the Maxwell-like wave equations

\begin{eqnarray*}
\frac{\partial E}{\partial t} & = & curl B \\
\frac{\partial B}{\partial t} & = & curl E
\end{eqnarray*}

where $E$ and $B$ are symmetric, traceless, div-free (STD) matrices

We choose $E \in H(cc)$, $B \in H(cd)$, and $v \in H(\operatorname{div})$. Then

$$
\left< \operatorname{curl} E, \delta B^T \right>  \quad \text{and} \quad
\left< \operatorname{div} B, \delta v \right> 
$$
are well defined.

using hyperbolic evolution for the divergence constraint we obtain the wave equation


$$
\frac{d}{dt} \left( \begin{array}{c} E \\ v \\ B \end{array} \right) = 
\left( \begin{array}{ccc} 
 & & C^T \\
 & & D \\
 -C & -D^T & 
\end{array} \right)
\left( \begin{array}{c} E \\ v \\ B \end{array} \right) 
$$

In [ ]:
from ngsolve import *
from ngsolve.webgui import Draw
mesh = Mesh(unit_cube.GenerateMesh(maxh=0.1))

In [ ]:
n = specialcf.normal(3)

def CurlTHcc2Hcd(E,dH):
    return InnerProduct(curl(E).trans, dH)*dx \
       +InnerProduct(Cross(E*n, n), dH*n)*dx(element_boundary= True)

def DivHcdHd(B,dv):
    return div(B)*dv*dx - B*n*n * dv*n * dx(element_boundary= True)

In [ ]:
order = 2

fescc = HCurlCurl(mesh, order=order)
fescd = HCurlDiv(mesh, order=order)
fesd = HDiv(mesh, order=order, RT=True)

E, dE = fescc.TnT()
v, dv = fesd.TnT()
B, dB = fescd.TnT()

In [ ]:
bfcurlT = BilinearForm(CurlTHcc2Hcd(E, dB)).Assemble()
bfdiv = BilinearForm(DivHcdHd(B, dv)).Assemble()

In [ ]:
for e in mesh.edges:
    for dof in fescc.GetDofNrs(e):
        fescc.couplingtype[dof] = COUPLING_TYPE.WIREBASKET_DOF
        
massE = BilinearForm(InnerProduct(E,dE)*dx, condense=True)
preE = Preconditioner(massE, "bddc", block=True, blocktype="edgepatch")
massE.Assemble()
matE = massE.mat
# preE = matE.CreateBlockSmoother(fescc.CreateSmoothingBlocks(blocktype="edgepatch", eliminate_internal=True), GS=False)

massEinvSchur = CGSolver (matE, preE)
ext = IdentityMatrix()+massE.harmonic_extension
extT = IdentityMatrix()+massE.harmonic_extension_trans
massEinv =  ext @ massEinvSchur @ extT + massE.inner_solve

In [ ]:
massB = BilinearForm(InnerProduct(B,dB)*dx, condense=True).Assemble()
matB = massB.mat
preB = matB.CreateBlockSmoother(fescd.CreateSmoothingBlocks(blocktype="facepatch", eliminate_internal=True), GS=False)
# preH = matH.CreateSmoother(fescd.FreeDofs(True), GS=False)

massBinvSchur = CGSolver (matB, preB)
ext = IdentityMatrix()+massB.harmonic_extension
extT = IdentityMatrix()+massB.harmonic_extension_trans
massBinv =  ext @ massBinvSchur @ extT + massB.inner_solve

In [ ]:
massv = BilinearForm(InnerProduct(v,dv)*dx, condense=True).Assemble()
matv = massv.mat
# prev = matv.CreateSmoother(fesd.FreeDofs(True), GS=False)
prev = matv.CreateBlockSmoother(fesd.CreateSmoothingBlocks(blocktype="facepatch", eliminate_internal=True), GS=False)

massvinvSchur = CGSolver (matv, prev)
ext = IdentityMatrix()+massv.harmonic_extension
extT = IdentityMatrix()+massv.harmonic_extension_trans
massvinv =  ext @ massvinvSchur @ extT + massv.inner_solve

In [ ]:
gfE = GridFunction(fescc)
gfB = GridFunction(fescd)
gfv = GridFunction(fesd)

In [ ]:
# initial conditions ....
gfB.vec[:] = 0.0
gfE.vec[:] = 0.0
gfv.vec[:] = 0.0

peak = exp(-((x-0.5)**2+(y-0.5)**2+(z-0.5)**2)/ 0.2**2 )
gfE.Set ( ((peak, 0,0), (0,0,0), (0,0,-peak) ))

In [ ]:
t = 0
tend = 10
dt = 5e-3
scene = Draw(Norm(gfB), mesh, clipping = { "vec" : (0,0,-1) }, settings={"Objects":{"Clipping Plane":True}})
energytrace = []
with TaskManager(): 
    while t < tend:
        gfE.vec.data += -dt * massEinv@bfcurlT.mat.T * gfB.vec
        gfv.vec.data += -dt * massvinv@bfdiv.mat * gfB.vec

        hv = bfcurlT.mat * gfE.vec + bfdiv.mat.T * gfv.vec
        gfB.vec.data += dt * massBinv * hv
        scene.Redraw()
    
        energytrace.append (Integrate ( Norm (Trace(gfE)), mesh ))
        t += dt

In [ ]:
import matplotlib.pyplot as plt
plt.plot (energytrace)